In [1]:
import pandas as pd
import numpy as np
import fileinput, sys

In [2]:
champ = 'Vayne'

In [3]:
df = pd.read_csv(f"data/{champ}.txt")

In [4]:
data = df.copy()

In [5]:
def dropOutliers(df):
    df.drop(data[data['hadAfkTeammate'] > 0].index, inplace=True)
    df.drop(data[(data['gameEndedInSurrender'] | data['gameEndedInSurrender']) & (data['gameDuration'] < 1350)].index, inplace=True)

In [6]:
dropOutliers(data)

In [7]:
def returnParams(df: pd.DataFrame):
    params = {}
    weight = 4
    relevantData = df.describe()
    meanScoreCreeps = relevantData.loc['mean']['scoreCreeps'] / (relevantData.loc['mean']['gameDuration'] / 60)
    meanDmg = relevantData.loc['mean']['totalDamageDealtToChampions'] / (relevantData.loc['mean']['gameDuration'] / 60)
    params['kills'] = 1/weight, 0, relevantData.loc['75%']['kills']
    params['deaths'] = 1/weight, 10, 0
    params['scoreCreeps'] = 1/weight, round(meanScoreCreeps * 0.6, 1), round(meanScoreCreeps * 1.5, 1)
    params['totalDamageDealtToChampions'] = 1/weight, round(meanDmg * 0.6), round(meanDmg * 1.5)
    # tuple(weigth, minPerformorance, maxPerformance)

    return params

In [8]:
params = returnParams(data)

In [25]:
def updateParams(champ, params: dict):
    fi = fileinput.input('test.txt', inplace=True)
    try:
        for line in fi:
            if line.find(champ) != -1:
                line = f'''Vayne;{';'.join(f"{k},{','.join(map(str, v))}" for k, v in params.items())}\n'''
            sys.stdout.write(line)
    except:
        fi.close()
    fi.close()